## <u>Online interactive Jupyter notebook as a supplement for:</u>

# Biomarker Set Identification for Post-Treatment Lyme Disease

**Daniel J.B. Clarke<sup>1</sup>, Alison W. Rebman<sup>2</sup>, Jinshui Fan<sup>2</sup>, Mark J. Soloski<sup>2</sup>, John N. Aucott<sup>2,\*</sup>, Avi Ma’ayan<sup>1,\*</sup>**

<sup>1</sup> Department of Pharmacological Sciences; Mount Sinai Center for Bioinformatics; Icahn School of Medicine at Mount Sinai, One Gustave L. Levy Place, Box 1603, New York, NY 10029, USA

<sup>2</sup> Lyme Disease Research Center, Division of Rheumatology, Department of Medicine, Johns Hopkins University School of Medicine, Baltimore, MD, United States

\* To whom correspondence should be addressed: jaucott2@jhmi.edu; avi.maayan@mssm.edu 


### Install dependencies

```bash
cat > requirements.txt << EOF
imbalanced-learn
maayanlab_bioinformatics@git+https://github.com/MaayanLab/maayanlab-bioinformatics.git@v0.5.1
matplotlib_venn
matplotlib
numpy
pandas
plotly
requests
seaborn
sklearn
supervenn
EOF

pip3 install -r requirements.txt

cat > setup.R << EOF
install.packages("R.utils")
install.packages("RCurl")

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")
BiocManager::install("limma")
BiocManager::install("statmod")
BiocManager::install("edgeR")
EOF

Rscript setup.R
```

In [ ]:
import json
import pathlib
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')
from matplotlib_venn import venn3
from supervenn import supervenn
from collections import OrderedDict
from IPython.display import display, Markdown
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedShuffleSplit, permutation_test_score, train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix, roc_curve, precision_recall_curve, confusion_matrix, auc, roc_auc_score, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from maayanlab_bioinformatics.harmonization import ncbi_genes_lookup
from maayanlab_bioinformatics.harmonization import transcripts_to_genes
from maayanlab_bioinformatics.harmonization import ncbi_genes_lookup
from maayanlab_bioinformatics.normalization import zscore_normalize, quantile_normalize, log2_normalize, filter_by_expr, filter_by_var
from maayanlab_bioinformatics.dge import limma_voom_differential_expression
from maayanlab_bioinformatics.api import EnrichrUserList
from maayanlab_bioinformatics.parse import gmt_read_dict

_lookup = ncbi_genes_lookup()
lookup = lambda gene: _lookup(gene, gene).upper()

In [ ]:
random_state = 42
data = pathlib.Path('data')
figures = pathlib.Path('figures')
figures.mkdir(exist_ok=True)

## LM2/LM3 Clinical Metadata Preparation

Construct unified metadata dataframes for LM2 & LM3

In [ ]:
def apply_desc(df, df_desc):
    ''' Ensure types match descriptors '''
    for column, kind in df_desc.items():
        if kind == 'categorical':
            df[column] = df[column].astype(float).dropna().astype(str)
        elif kind in {'label', 'timepoint'}:
            df[column] = df[column].dropna().astype(str)
        elif kind == 'continuous':
            df[column] = df[column].astype(float)
    return df

df_lm2_clinical = pd.read_csv(
    data / 'LM_Slice2_Slice3_data/LM2_Clinical_Entrez.csv', index_col=0
)
df_lm2_clinical_desc = {
    'orig_id': 'label',
    'time_point': 'label',
    'nebnext_index_read': 'label',
    'date': 'label',
    'batch': 'categorical',
    'pid': 'id',
    'visit': 'timepoint',
    'season_yr': 'categorical',
    'gender': 'categorical',
    'age': 'continuous',
    'hispa_dg': 'categorical',
    'race1_dg': 'categorical',
    'race2_dg': 'categorical',
    'hrsanti': 'categorical',
    'a1msd_fa': 'categorical',
    'durat': 'continuous',
    'durat_tx': 'continuous',
    'dissem': 'categorical',
    'rashsize': 'continuous',
    'elres_l2': 'categorical',
    'wbmto_l2': 'continuous',
    'wbgto_l2': 'continuous',
    'sero1': 'categorical',
    'sero2': 'categorical',
    'serogrp': 'categorical',
    'ibis_blood': 'categorical',
    'ibis_skin': 'categorical',
    'meno_stat': 'categorical',
    'newld_fh': 'categorical',
    'tempf_pe': 'continuous',
    'pulse_pe': 'continuous',
    'sbpre_pe': 'continuous',
    'dbpre_pe': 'continuous',
    'ast_l1': 'continuous',
    'alt_l1': 'continuous',
    'alk_l1': 'continuous',
    'lymph_l1': 'continuous',
    'wbc_l1': 'continuous',
    'gran_l1': 'continuous',
    'mono_l1': 'continuous',
    'eos_l1': 'continuous',
    'strev_fh': 'categorical',
    'numb_plqs': 'categorical',
    'sstot_sa': 'continuous',
    'pqapi_sa': 'continuous',
    'pqspi_sa': 'continuous',
    'pqtpi_sa': 'continuous',
    'ditot_sa': 'continuous',
    'sqtot_sa': 'continuous',
    'letot_sa': 'continuous',
    'pf_nbs': 'continuous',
    'rp_nbs': 'continuous',
    'bp_nbs': 'continuous',
    'gh_nbs': 'continuous',
    'vt_nbs': 'continuous',
    'sf_nbs': 'continuous',
    're_nbs': 'continuous',
    'mh_nbs': 'continuous',
    'pcs': 'continuous',
    'mcs': 'continuous',
    'type_subject': 'categorical',
}
df_lm2_clinical = apply_desc(df_lm2_clinical, df_lm2_clinical_desc)

df_lm2_clinical['dataset'] = 'lm2'
df_lm2_clinical.loc[df_lm2_clinical['type_subject']=='1.0', 'dataset_type_subject'] = 'lm2-case'
df_lm2_clinical.loc[df_lm2_clinical['type_subject']=='0.0', 'dataset_type_subject'] = 'lm2-ctrl'

df_lm3_clinical = pd.read_csv(
    data / 'LM_Slice2_Slice3_data/LM3_CaseOnly_Clinical_Entrez.csv', index_col=0,
)
df_lm3_clinical_desc = {
    'pid': 'id',
    'batch': 'label',
    'enroll_yr': 'label',
    'gender': 'categorical',
    'age': 'continuous',
    'hispa_dg': 'categorical',
    'race1_dg': 'categorical',
    'race2_dg': 'categorical',
    'time_en': 'continuous',
    'rash_mr': 'categorical',
    'flulike_mr': 'categorical',
    'neuro_mr': 'categorical',
    'carditis_mr': 'categorical',
    'arthritis_mr': 'categorical',
    'cur_sxnum': 'continuous',
    'time_tx': 'continuous',
    'ax_cur': 'categorical',
    'ax_tot': 'continuous',
    'pretx_inax': 'categorical',
    'st_exp': 'categorical',
    'elres_l2': 'categorical',
    'wbmto_l2': 'continuous',
    'wbgto_l2': 'continuous',
    'serogrp': 'categorical',
    'tempf_pe': 'continuous',
    'plssi_pe': 'continuous',
    'sbpsi_pe': 'continuous',
    'dbpsi_pe': 'continuous',
    'ast_l1': 'continuous',
    'alt_l1': 'continuous',
    'alk_l1': 'continuous',
    'lymph_l1': 'continuous',
    'wbc_l1': 'continuous',
    'gran_l1': 'continuous',
    'mono_l1': 'continuous',
    'eos_l1': 'continuous',
    'sstot_sa': 'continuous',
    'pqapi_sa': 'continuous',
    'pqspi_sa': 'continuous',
    'pqtpi_sa': 'continuous',
    'ditot_sa': 'continuous',
    'sqtot_sa': 'continuous',
    'letot_sa': 'continuous',
    'pf_nbs': 'continuous',
    'rp_nbs': 'continuous',
    'bp_nbs': 'continuous',
    'gh_nbs': 'continuous',
    'vt_nbs': 'continuous',
    'sf_nbs': 'continuous',
    're_nbs': 'continuous',
    'mh_nbs': 'continuous',
    'pcs': 'continuous',
    'mcs': 'continuous',
    'type_subject': 'categorical',
    'visit': 'timepoint',
}
df_lm3_clinical = apply_desc(df_lm3_clinical, df_lm3_clinical_desc)

df_lm3_clinical['dataset'] = 'lm3'
df_lm3_clinical['dataset_type_subject'] = 'lm3-case'

equivalent = [
    dict(id='dbp_dbp', lm2='dbpre_pe', lm3='dbpsi_pe'),
    dict(id='sbp_sbp', lm2='sbpre_pe', lm3='sbpsi_pe'),
    dict(id='pulse', lm2='pulse_pe', lm3='plssi_pe'),
    dict(id='time_till_treatment', lm2='durat_tx', lm3='time_tx'),
    dict(id='time_till_enrollment', lm2='durat', lm3='time_en'),
]
actually_different = [
    'visit',
    'batch',
    'pid',
]
certainly_different = [
    dict(lm2='meno_stat'),
    dict(lm3='rash_mr'),
    dict(lm3='carditis_mr'),
    dict(lm3='arthritis_mr'),
    dict(lm3='flulike_mr'),
]

for r in equivalent:
    df_lm2_clinical[r['id']] = df_lm2_clinical[r['lm2']]
    df_lm3_clinical[r['id']] = df_lm3_clinical[r['lm3']]

for c in actually_different:
    df_lm2_clinical.rename({ c: c+'-lm2' }, inplace=True, axis=1)
    df_lm3_clinical.rename({ c: c+'-lm3' }, inplace=True, axis=1)
    
common_clinical_columns = set(df_lm2_clinical.columns) & set(df_lm3_clinical.columns)
disjoint_clinical_columns = set(df_lm2_clinical.columns) ^ set(df_lm3_clinical.columns)

inner_clinical_columns = set(common_clinical_columns) \
  - set(actually_different) \
  | {r['id'] for r in equivalent}

outer_clinical_columns_lm2 = set(common_clinical_columns) \
  - set(actually_different) \
  | {r['id'] for r in equivalent} \
  | {c + '-lm2' for c in actually_different }
    
outer_clinical_columns_lm3 = set(common_clinical_columns) \
  - set(actually_different) \
  | {r['id'] for r in equivalent} \
  | {c + '-lm3' for c in actually_different }

df_lm23_inner_clinical = pd.concat([
    df_lm2_clinical[inner_clinical_columns],
    df_lm3_clinical[inner_clinical_columns],
], axis=0)
df_lm23_inner_clinical

df_lm23_outer_clinical = pd.concat([
    df_lm2_clinical,
    df_lm3_clinical,
], axis=0)
df_lm23_outer_clinical

In [ ]:
# obtain boolean masks for different categories
m_ptlds = df_lm23_outer_clinical['dataset_type_subject']=="lm3-case"
m_healthy = df_lm23_outer_clinical['dataset_type_subject']=="lm2-ctrl"
m_acute = (df_lm23_outer_clinical['visit-lm2'] == '1') & (df_lm23_outer_clinical['dataset_type_subject'] == 'lm2-case')
m_recovery = (df_lm23_outer_clinical['visit-lm2'] != '1') & (df_lm23_outer_clinical['dataset_type_subject'] == 'lm2-case')

## LM2/LM3 RNAseq Data Preparation

In [ ]:
def do_pca(df, with_ratio=False):
    pca = PCA()
    pca.fit(df.T)
    return pd.DataFrame(
        pca.transform(df.T),
        index=df.columns,
        columns=[
            'PC-{}{}'.format(n, ' ({:.3f}%)'.format(_r*100) if with_ratio else '')
            for n, _r in enumerate(pca.explained_variance_ratio_, start=1)
        ]
    )

In [ ]:
# healthy/acute from lm2
df_lm2_rnaseq = pd.read_csv(data / 'LM_Slice2_Slice3_data/LM2_Genetic_Entrez.csv', index_col=0)
df_lm2_genetic = transcripts_to_genes(df_lm2_rnaseq, strategy='sum')
df_lm2_genetic.index = df_lm2_genetic.index.map(lookup)
df_lm2_genetic_norm = df_lm2_genetic.loc[:, ~m_recovery]
df_lm2_genetic_norm = filter_by_expr(
    df_lm2_genetic_norm,
    design=df_lm2_clinical.loc[df_lm2_genetic.columns, 'type_subject'].astype(float).astype(int),
)
df_lm2_genetic_norm = log2_normalize(df_lm2_genetic_norm)
df_lm2_genetic_norm = zscore_normalize(df_lm2_genetic_norm.T).T
df_lm2_genetic_norm = quantile_normalize(df_lm2_genetic_norm)
df_lm2_genetic_norm_pca = do_pca(df_lm2_genetic_norm)

# ptld from lm3
df_lm3_rnaseq = pd.read_csv(data / 'LM_Slice2_Slice3_data/LM3_CaseOnly_Genetic_Entrez.csv', index_col=0)
df_lm3_genetic = transcripts_to_genes(df_lm3_rnaseq, strategy='sum')
df_lm3_genetic.index = df_lm3_genetic.index.map(lookup)
df_lm3_genetic_norm = df_lm3_genetic
df_lm3_genetic_norm = filter_by_expr(
    df_lm3_genetic_norm,
    design=df_lm3_clinical.loc[df_lm3_genetic.columns, 'type_subject'].astype(float).astype(int),
)
df_lm3_genetic_norm = log2_normalize(df_lm3_genetic_norm)
df_lm3_genetic_norm = quantile_normalize(df_lm3_genetic_norm)
df_lm3_genetic_norm = zscore_normalize(df_lm3_genetic_norm.T).T
df_lm3_genetic_norm_pca = do_pca(df_lm3_genetic_norm)

# joint lm2/lm3
df_lm23_genetic = pd.concat([df_lm2_genetic.loc[:, ~m_recovery], df_lm3_genetic], axis=1)
df_lm23_genetic_norm = df_lm23_genetic
df_lm23_genetic_norm = filter_by_expr(
    df_lm23_genetic,
    design=df_lm23_inner_clinical['dataset_type_subject'].map(
        lambda d: {
            'lm2-ctrl': 0,
            'lm2-case': 1,
            'lm3-case': 2,
        }[d]
    ),
)
df_lm23_genetic_norm = log2_normalize(df_lm23_genetic_norm)
df_lm23_genetic_norm = quantile_normalize(df_lm23_genetic_norm)
df_lm23_genetic_norm = zscore_normalize(df_lm23_genetic_norm.T).T
df_lm23_genetic_norm_pca = do_pca(df_lm23_genetic_norm, with_ratio=True)

In [ ]:
df_lm23_outer_clinical.loc[m_ptlds, 'Cohort'] = f'PTLD (n={m_ptlds.sum()})'
df_lm23_outer_clinical.loc[m_healthy, 'Cohort'] = f'Healthy (n={m_healthy.sum()})'
df_lm23_outer_clinical.loc[m_acute, 'Cohort'] = f'Acute LD (n={m_acute.sum()})'
# df_lm23_outer_clinical.loc[m_recovery, 'Cohort'] = f'LD Convalescent (n={m_recovery.sum()})'

In [ ]:
fig = px.scatter(
    pd.concat([
        df_lm23_genetic_norm_pca,
        df_lm23_outer_clinical.loc[~m_recovery],
    ], axis=1),
    x=df_lm23_genetic_norm_pca.columns[0],
    y=df_lm23_genetic_norm_pca.columns[1],
    color='Cohort',
)
fig.update_layout(
    autosize=True,
    width=1000,
    height=600,
    legend={'itemsizing': 'constant'},
)
fig.write_image(figures/'fig1.svg')
fig.show()

In [ ]:
df_lm23_outer_clinical['time_en_yr'] = df_lm23_outer_clinical['time_en'] / 365
df_lm23_outer_clinical['6mo'] = df_lm23_outer_clinical.apply(lambda r: ('Acute LD' if r['type_subject'] == '1.0' else 'Healthy') if pd.isna(r['time_en_yr']) else ('PTLD >6mo' if r['time_en_yr'] > 0.5 else 'PTLD <6mo'), axis=1)
count_lookup = df_lm23_outer_clinical['6mo'].value_counts().to_dict()
df_lm23_outer_clinical['6mo'] = df_lm23_outer_clinical['6mo'].apply(lambda f: f"{f} ({count_lookup[f]})")

fig = px.scatter(
    pd.concat([
        df_lm23_genetic_norm_pca,
        df_lm23_outer_clinical.loc[~m_recovery],
    ], axis=1),
    x=df_lm23_genetic_norm_pca.columns[0],
    y=df_lm23_genetic_norm_pca.columns[1],
    color='6mo',
)
fig.update_layout(
    autosize=True,
    width=1000,
    height=600,
    legend={'itemsizing': 'constant'},
)
fig.write_image(figures/'figS1.svg')
fig.show()

## Differential Expression Analysis

In [ ]:
df_lm2_genetic_ctrl = df_lm2_genetic.loc[:, m_healthy]
df_lm2_genetic_case_v1 = df_lm2_genetic.loc[:, m_acute]
df_lm3_genetic_case = df_lm3_genetic
df_lm2_genetic_lm3_genetic = pd.concat([df_lm2_genetic, df_lm3_genetic_case], axis=1)

In [ ]:
df_lm2_genetic_ctrl__lm2_genetic_case_v1 = limma_voom_differential_expression(
    df_lm2_genetic_ctrl,
    df_lm2_genetic_case_v1,
    df_lm2_genetic_lm3_genetic,
    voom_design=True, filter_genes=True,
)

df_lm2_genetic_ctrl__lm3_genetic_case = limma_voom_differential_expression(
    df_lm2_genetic_ctrl,
    df_lm3_genetic_case,
    df_lm2_genetic_lm3_genetic,
    voom_design=True, filter_genes=True,
)

df_lm2_genetic_case_v1__lm3_genetic_case = limma_voom_differential_expression(
    df_lm2_genetic_case_v1,
    df_lm3_genetic_case,
    df_lm2_genetic_lm3_genetic,
    voom_design=True, filter_genes=True,
)

cutoff = 0.01

lm2_ctrl__lm2_case_v1_up = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm2_genetic_case_v1[
        ((df_lm2_genetic_ctrl__lm2_genetic_case_v1['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm2_genetic_case_v1['t'] > 0))
    ].index
}

lm2_ctrl__lm3_case_up = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm3_genetic_case[
        ((df_lm2_genetic_ctrl__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm3_genetic_case['t'] > 0))
    ].index
}

lm2_case_v1__lm3_case_up = {
    lookup(gene)
    for gene in df_lm2_genetic_case_v1__lm3_genetic_case[
        ((df_lm2_genetic_case_v1__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_case_v1__lm3_genetic_case['t'] > 0))
    ].index
}

lm2_ctrl__lm2_case_v1_dn = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm2_genetic_case_v1[
        ((df_lm2_genetic_ctrl__lm2_genetic_case_v1['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm2_genetic_case_v1['t'] < 0))
    ].index
}

lm2_ctrl__lm3_case_dn = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm3_genetic_case[
        ((df_lm2_genetic_ctrl__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm3_genetic_case['t'] < 0))
    ].index
}

lm2_case_v1__lm3_case_dn = {
    lookup(gene)
    for gene in df_lm2_genetic_case_v1__lm3_genetic_case[
        ((df_lm2_genetic_case_v1__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_case_v1__lm3_genetic_case['t'] < 0))
    ].index
}

In [ ]:
genesets = {
    'Acute LD Up': lm2_ctrl__lm2_case_v1_up,
    'PTLD Up': lm2_ctrl__lm3_case_up,
    'Acute LD Down': lm2_ctrl__lm2_case_v1_dn,
    'PTLD Down': lm2_ctrl__lm3_case_dn,
    'Acute LD => PTLD': lm2_case_v1__lm3_case_up,
    'Acute LD <= PTLD': lm2_case_v1__lm3_case_dn,
    'Acute LD + PTLD Up': lm2_ctrl__lm2_case_v1_up & lm2_ctrl__lm3_case_up,
    'Acute LD + PTLD Down': lm2_ctrl__lm2_case_v1_dn & lm2_ctrl__lm3_case_dn,
}

print('\n'.join([
    f"{label:20} {len(geneset)}"
    for label, geneset in genesets.items()
]))

## Enrichment Analysis

We submit the differentially expressed genes to [Enrichr](https://maayanlab.cloud/Enrichr/).

In [ ]:
# Original code to generate these
# userlists = {
#     label: EnrichrUserList(genes, description=label)
#     for label, genes in genesets.items()
# }

# The user lists in enrichr
userlists = {
  userlist.description: userlist
  for userlist in [
    EnrichrUserList.from_url('https://maayanlab.cloud/Enrichr/enrich?dataset=5c8c3715899dbaa6ce7aa17d3fe0e77d'),
    EnrichrUserList.from_url('https://maayanlab.cloud/Enrichr/enrich?dataset=4a58c5ae103e3fa93861d231a9718f54'),
    EnrichrUserList.from_url('https://maayanlab.cloud/Enrichr/enrich?dataset=1954a8136b6aa8c0b73b1cff30ad5280'),
    EnrichrUserList.from_url('https://maayanlab.cloud/Enrichr/enrich?dataset=00e156d32ab62844391abaf9e3b0b823'),
    EnrichrUserList.from_url('https://maayanlab.cloud/Enrichr/enrich?dataset=41c885f2b79be29e03211733ca32d137'),
  ]
}

In [ ]:
display(Markdown('\n'.join(
    f'- {label} [{len(userlist.genes)}]: <{userlist.link}>' for label, userlist in userlists.items()
)))

In [ ]:
libraries = [
    'GO_Biological_Process_2021',
    'KEGG_2021_Human',
    'WikiPathways_2019_Human',
    'Human_Phenotype_Ontology',
    'MGI_Mammalian_Phenotype_Level_4_2021',
    'GWAS_Catalog_2019',
]

enrichment = []
for label, userlist in userlists.items():
  for library in libraries:
    df = userlist[library]
    enrichment.append((label, library, df[df['adjusted_pvalue'] < 0.01]))

df_enrichment = pd.DataFrame([
    dict(row.to_dict(), geneset=geneset, library=library)
    for geneset, library, d in enrichment
    for i, row in d.iterrows()
]).set_index(['geneset', 'library', 'term'])
df_enrichment.to_csv(figures / 'TableS2.tsv', sep='\t')

In [ ]:
# From Enrichr Compressed Bar Chart Figure Appyter
#  https://appyters.maayanlab.cloud/Enrichr_compressed_bar_chart_figure/

import re
from matplotlib.ticker import MaxNLocator

artifact = re.compile(r'\s+\(?\w{2}:\w+\)?\s+')
annot_dict = {}

def enrichr_figure(all_terms,all_pvalues, all_adjusted_pvalues, plot_names, all_libraries, fig_format, bar_color): 
    
    # rows and columns depend on number of Enrichr libraries submitted 
    rows = []
    cols = []
    
    # Bar colors
    if bar_color!= 'lightgrey':
        bar_color_not_sig = 'lightgrey'
        edgecolor=None
        linewidth=0
    else:
        bar_color_not_sig = 'white'
        edgecolor='black'
        linewidth=1
    
    # If only 1 Enrichr library selected, make simple plot 
    if len(all_libraries)==1:
        #fig,axes = plt.subplots(1, 1,figsize=[8.5,6])
        plt.figure(figsize=(12,6))
        rows = [0]
        cols = [0]
        i = 0 
        bar_colors = [bar_color if (x < 0.05) else bar_color_not_sig for x in all_pvalues[i]]
        fig = sns.barplot(x=np.log10(all_pvalues[i])*-1, y=all_terms[i], palette=bar_colors, edgecolor=edgecolor, linewidth=linewidth)
        fig.axes.get_yaxis().set_visible(False)
        fig.set_title(all_libraries[i].replace('_',' '),fontsize=26)
        fig.set_xlabel('-Log10(p-value)',fontsize=25)
        fig.xaxis.set_major_locator(MaxNLocator(integer=True))
        fig.tick_params(axis='x', which='major', labelsize=20)
        if max(np.log10(all_pvalues[i])*-1)<1:
            fig.xaxis.set_ticks(np.arange(0, max(np.log10(all_pvalues[i])*-1), 0.1))
        for ii,annot in enumerate(all_terms[i]):
            if annot in annot_dict.keys():
                annot = annot_dict[annot]
            if all_adjusted_pvalues[i][ii] < 0.05:
                annot = '  *'.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))]) 
            else:
                annot = '  '.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))])

            title_start= max(fig.axes.get_xlim())/200
            fig.text(title_start,ii,annot,ha='left',wrap = True, fontsize = 26)
            fig.patch.set_edgecolor('black')  
            fig.patch.set_linewidth('2')
        
    
    # If there are an even number of Enrichr libraries below 6
    # Plots 1x2 or 2x2
    else:
        if len(all_libraries) % 2 == 0 and len(all_libraries) < 5:
                for i in range(0,int(len(all_libraries)/2)):    
                    rows = rows + [i]*2
                    cols = list(range(0,2))*int(len(all_libraries)/2)    
                fig, axes = plt.subplots(len(np.unique(rows)), len(np.unique(cols)),figsize=[7,int(2* len(np.unique(rows)))]) 
    
        
        # All other # of libraries 6 and above will have 3 columns and a flexible number of rows to accomodate all plots
        else:
            for i in range(0,int(np.ceil(len(all_libraries)/3))):
                rows = rows + [i]*3
                cols = list(range(0,3))*int(np.ceil(len(all_libraries)/3))
            fig, axes = plt.subplots(len(np.unique(rows)), len(np.unique(cols)),figsize=[8,int(2* len(np.unique(rows)))])
           
        # If final figure only has one row...
        if len(np.unique(rows))==1:
            for i,library_name in enumerate(all_libraries):
                bar_colors = [bar_color if (x < 0.05) else bar_color_not_sig for x in all_pvalues[i]]
                sns.barplot(x=np.log10(all_pvalues[i])*-1, y=all_terms[i],ax=axes[i], palette=bar_colors, edgecolor=edgecolor, linewidth=linewidth)
                axes[i].axes.get_yaxis().set_visible(False)
                axes[i].set_title(library_name.replace('_',' '),fontsize=36)
                axes[i].set_xlabel('-Log10(p-value)',fontsize=35)
                axes[i].xaxis.set_major_locator(MaxNLocator(integer=True))
                axes[i].tick_params(axis='x', which='major', labelsize=30)
                if max(np.log10(all_pvalues[i])*-1)<1:
                    axes[i].xaxis.set_ticks(np.arange(0, max(np.log10(all_pvalues[i])*-1), 0.1))
                for ii,annot in enumerate(all_terms[i]):
                    if annot in annot_dict.keys():
                        annot = annot_dict[annot]
                    if all_adjusted_pvalues[i][ii] < 0.05:
                        annot = '  *'.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))]) 
                    else:
                        annot = '  '.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))])

                    title_start= max(axes[i].axes.get_xlim())/200
                    axes[i].text(title_start,ii,annot,ha='left',wrap = True, fontsize = 36)
                    axes[i].patch.set_edgecolor('black')  
                    axes[i].patch.set_linewidth('2')

            plt.subplots_adjust(top=4.5, right = 4.7,wspace = 0.03,hspace = 0.2)


        # If the final figure has more than one row...
        else:


            for i,library_name in enumerate(all_libraries):
                bar_colors = [bar_color if (x < 0.05) else bar_color_not_sig for x in all_pvalues[i]]
                sns.barplot(x=np.log10(all_pvalues[i])*-1, y=all_terms[i],ax=axes[rows[i],cols[i]], palette=bar_colors, edgecolor=edgecolor, linewidth=linewidth)
                axes[rows[i],cols[i]].axes.get_yaxis().set_visible(False)
                axes[rows[i],cols[i]].set_title(library_name.replace('_',' '),fontsize=36)
                axes[rows[i],cols[i]].set_xlabel('-Log10(p-value)',fontsize=35)
                axes[rows[i],cols[i]].xaxis.set_major_locator(MaxNLocator(integer=True))
                axes[rows[i],cols[i]].tick_params(axis='x', which='major', labelsize=30)
                if max(np.log10(all_pvalues[i])*-1)<1:
                    axes[rows[i],cols[i]].xaxis.set_ticks(np.arange(0, max(np.log10(all_pvalues[i])*-1), 0.1))
                for ii,annot in enumerate(all_terms[i]):
                    if annot in annot_dict.keys():
                        annot = annot_dict[annot]
                    if all_adjusted_pvalues[i][ii] < 0.05:
                        annot = '  *'.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))]) 
                    else:
                        annot = '  '.join([annot, str(str(np.format_float_scientific(all_pvalues[i][ii],precision=2)))])

                    title_start = max(axes[rows[i],cols[i]].axes.get_xlim())/200
                    axes[rows[i],cols[i]].text(title_start, ii, re.sub(artifact, ' ', annot).strip(), ha='left',wrap = True,va='center', fontsize = 24)
                    axes[rows[i],cols[i]].patch.set_edgecolor('black')  
                    axes[rows[i],cols[i]].patch.set_linewidth('2')

            plt.subplots_adjust(top=4.8, right = 4.7,wspace = 0.03,hspace = 0.2)


        # If >6 libraries are chosen and is not a multiple of 3, delete empty plots
        if len(np.unique(rows))*len(np.unique(cols)) != len(all_libraries):
            diff = (len(np.unique(rows))*len(np.unique(cols))) - len(all_libraries)
            for i in range (1,int(diff+1)):
                fig.delaxes(axes[rows[-i]][cols[-i]])
    
    # Save results 
    for plot_name in plot_names:
        plt.savefig(str(figures/plot_name),format=fig_format, bbox_inches = 'tight')
    
    # Show plot 
    plt.show()

for label, userlist in userlists.items():
    print(label)
    all_terms = []
    all_pvalues = []
    all_adjusted_pvalues = []
    library_success = []
    for lib in libraries:
        try:
            df_lib_top = userlist[lib].iloc[:5]
            all_terms.append(df_lib_top['term'].tolist())
            all_pvalues.append(df_lib_top['pvalue'].tolist())
            all_adjusted_pvalues.append(df_lib_top['adjusted_pvalue'].tolist())
            library_success.append(lib)
        except:
            print('Error for ' + lib + ' library')
    #
    enrichr_figure(
        all_terms,
        all_pvalues,
        all_adjusted_pvalues,
        [f"fig2-3-{label.replace('/','-')}.pdf"],
        library_success,
        'pdf', 'tomato'
    )


### KEGG Enrichment

We look specifically the Herpes Enriched term in KEGG.

In [ ]:
res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human')
KEGG = gmt_read_dict(res.text.split('\n'))

In [ ]:
venn3([
    lm2_ctrl__lm2_case_v1_dn,
    set(KEGG['Herpes simplex virus 1 infection']),
    lm2_ctrl__lm3_case_up,
], [
    'Down Regulated Genes in Acute LD',
    'KEGG_2021_Human: Herpes simplex virus 1 infection',
    'Up Regulated Genes in PTLD',
])
plt.savefig(figures/'fig3.svg', bbox_inches='tight', pad_inches=0.2)

# Enrichr Signatures

We load geneset biomarkers implicated in other diseases.

In [ ]:
def prefix_keys(*args):
    *prefix, obj = args
    return { (*prefix, key): value for key, value in obj.items() }

def dict_union(*dicts):
    union = {}
    for d in dicts: union.update(d)
    return union

def union_signature(signatures):
    return set.union(*map(set, signatures.values()))

def gather_signatures(library, condition):
    q = {k for k,v in library.items() if condition(k)}
    display(q)
    return { k: { lookup(vv) for vv in library[k] } for k in q }


In [ ]:
# grab geneset-libraries from enrichr
res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GO_Biological_Process_2018')
GO = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Disease_Perturbations_from_GEO_up')
GEO_disease_up = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Microbe_Perturbations_from_GEO_up')
GEO_microbe_up = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=GWAS_Catalog_2019')
GWAS_2019 = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=DisGeNET')
DisGeNET = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=COVID-19_Related_Gene_Sets')
COV = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Jensen_DISEASES')
DISEASES = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Rare_Diseases_GeneRIF_Gene_Lists')
Rare_Diseases_GeneRIF_Gene_Lists = gmt_read_dict(res.text.split('\n'))

res = requests.get('https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Rare_Diseases_AutoRIF_Gene_Lists')
Rare_Diseases_AutoRIF_Gene_Lists = gmt_read_dict(res.text.split('\n'))

In [ ]:
signatures = dict_union(
    prefix_keys('Term',        None, 'GO_Biological_Process_2018',
                gather_signatures(GO, lambda k: k in {'inflammatory response (GO:0006954)', 'cellular response to molecule of bacterial origin (GO:0071219)'})),
    prefix_keys('Viral',       'Influenza', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: 'influenza' in k.lower())),
    prefix_keys('Viral',       'HIV', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: k == 'HIV-1 human PBMC GDS1449 microbe:221')),
    prefix_keys('Viral',       'COVID-19', 'COVID-19_Related_Gene_Sets',
                gather_signatures(COV, lambda k: k == 'COVID-19 patients PBMC up')),
    prefix_keys('Bacterial',   'Tuberculosis', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: 'tuberculosis' in k.lower() and 'human pbmc' in k.lower())),
    prefix_keys('Bacterial',   'Streptococcus', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: 'streptococcus' in k.lower() and 'human' in k.lower())),
    prefix_keys('Bacterial',   'Pneumococcal', 'DisGeNET',
                gather_signatures(DisGeNET, lambda k: 'pneumococcal' in k.lower())),
    prefix_keys('Bacterial',   'E. Coli', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: 'escherichia coli' in k.lower())),
    prefix_keys('Bacterial',   'E. Coli', 'Disease_Perturbations_from_GEO_up',
                gather_signatures(GEO_disease_up, lambda k: 'escherichia coli' in k.lower())),
    prefix_keys('Bacterial',   'Staphylococcus', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: 'staphylococcus' in k.lower() and 'human' in k.lower())),
    prefix_keys('Bacterial',   'Gonorrhoeae', 'Microbe_Perturbations_from_GEO_up',
                gather_signatures(GEO_microbe_up, lambda k: 'gonorrhoeae' in k.lower() and 'pbmc' in k.lower())),
    prefix_keys('Bacterial',   'Sepsis', 'Disease_Perturbations_from_GEO_up',
                gather_signatures(GEO_disease_up, lambda k: 'sepsis' in k.lower())),
    prefix_keys('Bacterial',   'Sepsis', 'DisGeNET',
                gather_signatures(DisGeNET, lambda k: 'sepsis' in k.lower())),
    prefix_keys('Spirochetes', 'Syphilis', 'DisGeNET',
                gather_signatures(DisGeNET, lambda k: 'syphilis' in k.lower())),
    prefix_keys('Spirochetes', 'Syphilis', 'Jensen_DISEASES',
                gather_signatures(DISEASES, lambda k: 'syphilis' in k.lower())),
    prefix_keys('Spirochetes', 'Leptospirosis', 'DisGeNET',
                gather_signatures(DisGeNET, lambda k: 'leptospirosis' in k.lower())),
    prefix_keys('Spirochetes', 'Leptospirosis', 'Jensen_DISEASES',
                gather_signatures(DISEASES, lambda k: 'leptospirosis' in k.lower())),
    prefix_keys('Spirochetes', 'Yaws', 'Jensen_DISEASES',
                gather_signatures(DISEASES, lambda k: 'yaws' in k.lower())),
    prefix_keys('Spirochetes', 'Relapsing Fever', 'DisGeNET',
                gather_signatures(DisGeNET, lambda k: 'relapsing fever' in k.lower())),
    prefix_keys('Spirochetes', 'Relapsing Fever', 'Jensen_DISEASES',
                gather_signatures(DISEASES, lambda k: 'relapsing fever' in k.lower())),
    prefix_keys('Spirochetes', 'Periodontal Disease', 'DisGeNET',
                gather_signatures(DisGeNET, lambda k: 'periodontal disease' in k.lower())),
    prefix_keys('Spirochetes', 'Bejel', 'Rare_Diseases_GeneRIF_Gene_Lists',
                gather_signatures(Rare_Diseases_GeneRIF_Gene_Lists, lambda k: 'bejel' in k.lower())),
    prefix_keys('Spirochetes', 'Bejel', 'Rare_Diseases_AutoRIF_Gene_Lists',
                gather_signatures(Rare_Diseases_AutoRIF_Gene_Lists, lambda k: 'bejel' in k.lower())),
    prefix_keys('Spirochetes', 'Pinta', 'Rare_Diseases_AutoRIF_Gene_Lists',
                gather_signatures(Rare_Diseases_AutoRIF_Gene_Lists, lambda k: 'pinta' in k.lower())),

)
df_signatures = pd.DataFrame([
    {
        'category': category,
        'disease': disease,
        'library': library,
        'term': term,
        'n_genes': len(genes)
    }
    for (category, disease, library, term), genes in signatures.items()
])
df_signatures.to_csv(figures / 'TableS1.tsv', sep='\t')
df_signatures

In [ ]:
lm_universe = set.union(
    lm2_ctrl__lm2_case_v1_up,
    lm2_ctrl__lm3_case_up,
    lm2_ctrl__lm2_case_v1_dn,
    lm2_ctrl__lm3_case_dn,
)
inflammatory_term = {gene for (category, disease, library, term), genes in signatures.items() if category == 'Term' and term == 'inflammatory response (GO:0006954)' for gene in genes}
bacteria_term = {gene for (category, disease, library, term), genes in signatures.items() if category == 'Term' and term == 'cellular response to molecule of bacterial origin (GO:0071219)' for gene in genes}
bacterial_union = {gene for (category, disease, library, term), genes in signatures.items() if category == 'Bacterial' for gene in genes}
viral_union = {gene for (category, disease, library, term), genes in signatures.items() if category == 'Viral' for gene in genes}
spirochetes_union = {gene for (category, disease, library, term), genes in signatures.items() if category == 'Spirochetes' for gene in genes}

## Visualization

In [ ]:
sets = OrderedDict([
  ('Up Regulated Genes in Acute LD', lm2_ctrl__lm2_case_v1_up),
  ('Up Regulated Genes in PTLD', lm2_ctrl__lm3_case_up),
  ('Down Regulated Genes in Acute LD', lm2_ctrl__lm2_case_v1_dn),
  ('Down Regulated Genes in PTLD', lm2_ctrl__lm3_case_dn),
  ('Inflammatory Response (GO:0006954)', inflammatory_term & lm_universe),
  ('Cellular Response to Molecule of Bacterial Origin (GO:0071219)', bacteria_term & lm_universe),
  ('Bacterial Enrichr Gene Sets', bacterial_union & lm_universe),
  ('Viral Enrichr Gene Sets', viral_union & lm_universe),
  ('Spirochete Enrichr Gene Sets', spirochetes_union & lm_universe),
])

with plt.style.context('bmh'):
    plt.figure(figsize=(20,10))
    supervenn(
        *zip(*((v, k) for k, v in sets.items())),
        sets_ordering='minimize gaps',
        widths_minmax_ratio=0.1,
        min_width_for_annotation=10,
    )
    plt.savefig(figures / 'fig4.svg')

In [ ]:
consensus_up = (lm2_ctrl__lm2_case_v1_up & lm2_ctrl__lm3_case_up) - bacterial_union - viral_union - spirochetes_union
consensus_dn = (lm2_ctrl__lm2_case_v1_dn & lm2_ctrl__lm3_case_dn) - bacterial_union - viral_union - spirochetes_union
divergent = ((lm2_ctrl__lm2_case_v1_up & lm2_ctrl__lm3_case_dn) | (lm2_ctrl__lm2_case_v1_dn & lm2_ctrl__lm3_case_up)) - bacterial_union - viral_union - spirochetes_union
len(consensus_up), len(consensus_dn), len(divergent)

In [ ]:
relevant = list(consensus_up | consensus_dn | divergent)
len(relevant)

## Feature Selection

We attempt to reduce the number of genes in this set.

In [ ]:
d = pd.concat([
    df_lm2_genetic_lm3_genetic.loc[relevant, :].T,
    df_lm23_inner_clinical[['dataset_type_subject']],
], axis=1).reset_index().set_index(['dataset_type_subject', 'index'])
score = (d.var() / d.groupby(level=0).var()).mean().sort_values(ascending=False)
sns.histplot(score)
plt.ylabel('Genes')
plt.xlabel('mean(total_variance / inter_group_variance)')
plt.show()
display(score[score>3])
var_selected_genes = list(score[score>3].index)

In [ ]:
task_defs = [
    (
        'Lyme v Healthy',
        m_acute|m_ptlds, m_healthy,
    ),
    (
        'Acute LD v Healthy',
        m_acute, m_healthy,
    ),
    (
        'PTLD v Healthy',
        m_ptlds, m_healthy,
    ),
    (
        'Acute LD v PTLD',
        m_acute, m_ptlds,
    ),
]

tasks = []
for label, pos, neg in task_defs:
    X = df_lm2_genetic_lm3_genetic.loc[relevant, neg|pos].T
    y = pd.Series(np.in1d(X.index, pos[pos].index).astype(int), index=X.index)
    tasks.append((label, X, y))

In [ ]:
importances = {}
for label, X, y in tasks:
    clf = LogisticRegression()#max_iter=800)
    clf.fit(X, y)
    ret = permutation_importance(clf, X, y, n_repeats=50, random_state=random_state)
    importance = pd.Series(ret.importances_mean, index=X.columns).sort_values()
    sns.histplot(importance); plt.show()
    importances[label] = importance

In [ ]:
cutoff = 0.3
consensus = {k: (v-v.mean())/v.std() for k, v in importances.items()}
consensus.update(var=(score-score.mean())/score.std())
df_consensus = pd.concat(consensus.values(), axis=1).dropna()
df_consensus.columns = consensus.keys()
display(df_consensus)
df_consensus_agg = df_consensus.mean(axis=1).sort_values()
sns.histplot(df_consensus_agg)
plt.vlines([cutoff*df_consensus_agg.std()], ymin=0,ymax=5, color='black')
plt.yscale('log')
plt.xlabel('Average score')
plt.show()

In [ ]:
df_consensus_selected = df_consensus_agg[df_consensus_agg > cutoff * df_consensus_agg.std()]
df_consensus_selected.to_frame('Average Score')

In [ ]:
final_consensus = df_consensus_selected.index

## Holdout

Though these genes are relevant, a feature selection bias could arrise, thus we'll do this feature selection with held out samples as to assert generalizability of our model. We will validate against these samples which will not be used for feature selection or training.

In [ ]:
df_lm2_genetic_ctrl_train, df_lm2_genetic_ctrl_test = map(lambda v: v.T, train_test_split(df_lm2_genetic_ctrl.T, random_state=random_state))
df_lm2_genetic_case_v1_train, df_lm2_genetic_case_v1_test = map(lambda v: v.T, train_test_split(df_lm2_genetic_case_v1.T, random_state=random_state))
df_lm3_genetic_case_train, df_lm3_genetic_case_test = map(lambda v: v.T, train_test_split(df_lm3_genetic_case.T, random_state=random_state))
df_lm2_genetic_lm3_genetic_train, df_lm2_genetic_lm3_genetic_test = pd.concat([df_lm2_genetic_ctrl_train, df_lm2_genetic_case_v1_train, df_lm3_genetic_case_train], axis=1), pd.concat([df_lm2_genetic_ctrl_test, df_lm2_genetic_case_v1_test, df_lm3_genetic_case_test], axis=1)

In [ ]:
print('\n'.join([
    f"Holdout from {df_lm2_genetic_lm3_genetic.shape=}",
    f"type   {'train'                                 :15} test",
    f"ctrl   {repr(df_lm2_genetic_ctrl_train.shape)   :15} {df_lm2_genetic_ctrl_test.shape}",
    f"acute  {repr(df_lm2_genetic_case_v1_train.shape):15} {df_lm2_genetic_case_v1_test.shape}",
    f"ptld   {repr(df_lm3_genetic_case_train.shape)   :15} {df_lm3_genetic_case_test.shape}",
]))

In [ ]:
df_lm2_genetic_ctrl__lm2_genetic_case_v1 = limma_voom_differential_expression(
    df_lm2_genetic_ctrl_train,
    df_lm2_genetic_case_v1_train,
    df_lm2_genetic_lm3_genetic_train,
    voom_design=True, filter_genes=True,
)

df_lm2_genetic_ctrl__lm3_genetic_case = limma_voom_differential_expression(
    df_lm2_genetic_ctrl_train,
    df_lm3_genetic_case_train,
    df_lm2_genetic_lm3_genetic_train,
    voom_design=True, filter_genes=True,
)

df_lm2_genetic_case_v1__lm3_genetic_case = limma_voom_differential_expression(
    df_lm2_genetic_case_v1_train,
    df_lm3_genetic_case_train,
    df_lm2_genetic_lm3_genetic_train,
    voom_design=True, filter_genes=True,
)

cutoff = 0.01

lm2_ctrl__lm2_case_v1_up = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm2_genetic_case_v1[
        ((df_lm2_genetic_ctrl__lm2_genetic_case_v1['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm2_genetic_case_v1['t'] > 0))
    ].index
}

lm2_ctrl__lm3_case_up = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm3_genetic_case[
        ((df_lm2_genetic_ctrl__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm3_genetic_case['t'] > 0))
    ].index
}

lm2_ctrl__lm2_case_v1_dn = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm2_genetic_case_v1[
        ((df_lm2_genetic_ctrl__lm2_genetic_case_v1['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm2_genetic_case_v1['t'] < 0))
    ].index
}

lm2_ctrl__lm3_case_dn = {
    lookup(gene)
    for gene in df_lm2_genetic_ctrl__lm3_genetic_case[
        ((df_lm2_genetic_ctrl__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_ctrl__lm3_genetic_case['t'] < 0))
    ].index
}

lm2_case_v1__lm3_case_up = {
    lookup(gene)
    for gene in df_lm2_genetic_case_v1__lm3_genetic_case[
        ((df_lm2_genetic_case_v1__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_case_v1__lm3_genetic_case['t'] > 0))
    ].index
}

lm2_case_v1__lm3_case_dn = {
    lookup(gene)
    for gene in df_lm2_genetic_case_v1__lm3_genetic_case[
        ((df_lm2_genetic_case_v1__lm3_genetic_case['adj.P.Val'] < cutoff) & (df_lm2_genetic_case_v1__lm3_genetic_case['t'] < 0))
    ].index
}

In [ ]:
lm_universe = set.union(
    lm2_ctrl__lm2_case_v1_up,
    lm2_ctrl__lm3_case_up,
    lm2_ctrl__lm2_case_v1_dn,
    lm2_ctrl__lm3_case_dn,
)
consensus_up = (lm2_ctrl__lm2_case_v1_up & lm2_ctrl__lm3_case_up) - bacterial_union - viral_union - spirochetes_union
consensus_dn = (lm2_ctrl__lm2_case_v1_dn & lm2_ctrl__lm3_case_dn) - bacterial_union - viral_union - spirochetes_union
divergent = ((lm2_ctrl__lm2_case_v1_up & lm2_ctrl__lm3_case_dn) | (lm2_ctrl__lm2_case_v1_dn & lm2_ctrl__lm3_case_up)) - bacterial_union - viral_union - spirochetes_union
relevant = list(consensus_up | consensus_dn | divergent)
display((len(consensus_up), len(consensus_dn), len(divergent)))
display(len(relevant))

In [ ]:
d = pd.concat([
    df_lm2_genetic_lm3_genetic_train.loc[relevant, :].T,
    df_lm23_inner_clinical[['dataset_type_subject']],
], axis=1).reset_index().set_index(['dataset_type_subject', 'index'])
score = (d.var() / d.groupby(level=0).var()).mean().sort_values(ascending=False)
sns.histplot(score)
plt.ylabel('Genes')
plt.xlabel('mean(total_variance / inter_group_variance)')
plt.show()
display(score[score>3])
var_selected_genes = list(score[score>3].index)

In [ ]:
train_tasks = []
for label, pos, neg in task_defs:
    X_train = df_lm2_genetic_lm3_genetic_train.loc[relevant, neg|pos].T
    y_train = X_train.index.to_series().isin(pos[pos].index).astype(int)
    X_test = df_lm2_genetic_lm3_genetic_test.loc[relevant, neg|pos].T
    y_test = X_test.index.to_series().isin(pos[pos].index).astype(int)
    train_tasks.append((label, X_train, y_train, X_test, y_test))

In [ ]:
importances = {}
for label, X_train, y_train, _X_test, _y_test in train_tasks:
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    ret = permutation_importance(clf, X_train, y_train, n_repeats=50, random_state=random_state)
    importance = pd.Series(ret.importances_mean, index=X_train.columns).sort_values()
    sns.histplot(importance); plt.show()
    importances[label] = importance

In [ ]:
cutoff = 0.3
consensus = {k: (v-v.mean())/v.std() for k, v in importances.items()}
consensus.update(var=(score-score.mean())/score.std())
df_consensus = pd.concat(consensus.values(), axis=1).dropna()
df_consensus.columns = consensus.keys()
display(df_consensus)
df_consensus_agg = df_consensus.mean(axis=1).sort_values()
sns.histplot(df_consensus_agg)
plt.vlines([cutoff*df_consensus_agg.std()], ymin=0,ymax=5, color='black')
plt.yscale('log')
plt.xlabel('Average score')

In [ ]:
df_consensus_selected = df_consensus_agg[df_consensus_agg > cutoff * df_consensus_agg.std()]
df_consensus_selected.to_frame('Average Score')

In [ ]:
from matplotlib_venn import venn2
venn2([set(df_consensus_selected.index), set(final_consensus)])

In [ ]:
features_label = 'Biomarker'
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
fig2, ((ax2_11, ax2_12), (ax2_21, ax2_22)) = plt.subplots(2, 2, figsize=(12, 12))
for (label, X_train, y_train, X_test, y_test), ax2_n in zip(train_tasks, (ax2_11, ax2_12, ax2_21, ax2_22)):
    # add determinism
    rs = np.random.RandomState(seed=random_state)
    clf = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
    # how consistent are these results across different stratified shuffle splits
    _score, _permutation_scores, pvalue = permutation_test_score(clf, X_train, y_train,
      n_permutations=200,
      cv=StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=rs),
      random_state=rs,
    )
    full_label = f"{features_label} {label} (p={pvalue:.3f})"
    # during training, we oversample to maximize utilization of the available
    #  samples but learn a balanced dataset
    rus = RandomOverSampler(random_state=rs)
    X_train_resamp, y_train_resamp = X_train, y_train
    X_train_resamp, y_train_resamp = rus.fit_resample(X_train, y_train)
    clf.fit(X_train_resamp, y_train_resamp)
    # during testing, we undersample such that we have a 50/50 prior
    #  because the actual distribution is not balanced and skewed to i.e. PTLD & Acute Lyme
    rus = RandomUnderSampler(random_state=rs)
    X_test_resamp, y_test_resamp = X_test, y_test
    X_test_resamp, y_test_resamp = rus.fit_resample(X_test, y_test)
    plot_confusion_matrix(clf, X_test_resamp, y_test_resamp, ax=ax2_n)
    ax2_n.set_title(full_label)
    plot_precision_recall_curve(clf, X_test_resamp, y_test_resamp, name=full_label, ax=ax2)
    plot_roc_curve(clf, X_test_resamp, y_test_resamp, name=full_label, ax=ax1)
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=1)
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=1)
fig.show()
fig2.show()
fig.savefig(figures/'fig5.svg', bbox_inches='tight', pad_inches=0.2)
fig2.savefig(figures/'fig5.svg', bbox_inches='tight', pad_inches=0.2)

In [ ]:
# KLHL11 and UTF1 are the best performers, and are highly correlated to PC-1 & PC-2

features_label = 'KLHL11-UTF1'
features = ['KLHL11', 'UTF1']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
fig2, ((ax2_11, ax2_12), (ax2_21, ax2_22)) = plt.subplots(2, 2, figsize=(12, 12))
for (label, X_train, y_train, X_test, y_test), ax2_n in zip(train_tasks, (ax2_11, ax2_12, ax2_21, ax2_22)):
    X_train = X_train.loc[:, features]
    X_test = X_test.loc[:, features]
    rs = np.random.RandomState(seed=random_state)
    clf = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
    # how consistent are these results across different stratified shuffle splits
    _score, _permutation_scores, pvalue = permutation_test_score(clf, X_train, y_train,
      n_permutations=200,
      cv=StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=rs),
      random_state=rs,
    )
    full_label = f"{features_label} {label} (p={pvalue:.3f})"
    # during training, we oversample to maximize utilization of the available
    #  samples but learn a balanced dataset
    rus = RandomOverSampler(random_state=rs)
    X_train_resamp, y_train_resamp = X_train, y_train
    X_train_resamp, y_train_resamp = rus.fit_resample(X_train, y_train)
    clf.fit(X_train_resamp, y_train_resamp)
    # during testing, we undersample such that we have a 50/50 prior
    #  because the actual distribution is not balanced and skewed to i.e. PTLD & Acute Lyme
    rus = RandomUnderSampler(random_state=rs)
    X_test_resamp, y_test_resamp = X_test, y_test
    X_test_resamp, y_test_resamp = rus.fit_resample(X_test, y_test)
    plot_confusion_matrix(clf, X_test_resamp, y_test_resamp, ax=ax2_n)
    ax2_n.set_title(full_label)
    plot_precision_recall_curve(clf, X_test_resamp, y_test_resamp, name=full_label, ax=ax2)
    plot_roc_curve(clf, X_test_resamp, y_test_resamp, name=full_label, ax=ax1)
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=1)
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=1)
fig.show()
fig2.show()
fig.savefig(figures/'figS3.svg', bbox_inches='tight', pad_inches=0.2)
fig2.savefig(figures/'figS3.svg', bbox_inches='tight', pad_inches=0.2)

## Supplemental Analysis: Single Biomarker Performance

We evaluate each individual biomarker on its performance as a standalone predictor for each classification task.

In [ ]:
results = []
for g in final_consensus:
    features = [g]
    for label, X, y in tasks:
        rs = np.random.RandomState(seed=random_state)
        X_train, X_test, y_train, y_test = train_test_split(X.loc[:, features], y, test_size=0.5, random_state=rs)        
        clf = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
        # during training, we oversample to maximize utilization of the available
        #  samples but learn a balanced dataset
        rus = RandomOverSampler(random_state=rs)
        X_train_resamp, y_train_resamp = rus.fit_resample(X_train, y_train)
        clf.fit(X_train_resamp, y_train_resamp)
        # during testing, we undersample such that we have a 50/50 prior
        #  because the actual distribution is not balanced and skewed to i.e. PTLD & Acute Lyme
        rus = RandomUnderSampler(random_state=rs)
        X_test_resamp, y_test_resamp = rus.fit_resample(X_test, y_test)

        # we capture these values for per-feature analysis
        fpr_train, tpr_train, _ = roc_curve(clf.predict(X_train_resamp), y_train_resamp)
        roc_score_train = auc(fpr_train, tpr_train)
        prec_train, rec_train, _ = precision_recall_curve(clf.predict(X_train_resamp), y_train_resamp)
        pr_score_train = average_precision_score(clf.predict(X_train_resamp), y_train_resamp)
        confusion_train = confusion_matrix(clf.predict(X_train_resamp), y_train_resamp)
        
        fpr_test, tpr_test, _ = roc_curve(clf.predict(X_test_resamp), y_test_resamp)
        roc_score_test = auc(fpr_test, tpr_test)
        prec_test, rec_test, _ = precision_recall_curve(clf.predict(X_test_resamp), y_test_resamp)
        pr_score_test = average_precision_score(clf.predict(X_test_resamp), y_test_resamp)
        confusion_test = confusion_matrix(clf.predict(X_test_resamp), y_test_resamp)

        results.append(dict(
            label=label, gene=g,
            train=dict(
                confusion=confusion_train,
                roc=dict(fpr=fpr_train, tpr=tpr_train, auc=roc_score_train),
                pr=dict(prec=fpr_train, rec=tpr_train, auc=pr_score_train),
            ),
            test=dict(
                confusion=confusion_test,
                roc=dict(fpr=fpr_test, tpr=tpr_test, auc=roc_score_test),
                pr=dict(prec=prec_test, rec=rec_test, auc=pr_score_test),
            )
        ))

In [ ]:
df_results = pd.DataFrame([
    dict(label=r['label'], gene=r['gene'], kind=kind, auc=r[kind]['roc']['auc'])
    for i, r in enumerate(results)
    for kind in ['train', 'test']
])

In [ ]:
fig=plt.figure(figsize=(16, 6))
sns.violinplot(data=df_results, x='gene',y='auc', hue='kind')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.clustermap(df_results[(df_results['kind']=='test')].pivot('gene', 'label', 'auc'))
plt.savefig(figures/'fig7.svg')